In [2]:
import pandas as pd
import gzip
import json
import numpy as np
import os
import ast

## Analisis de User_Items

In [3]:

# Ruta del archivo comprimido
file_path = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/users_items.json.gz'
# Definimos una función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(nombreArchivo):

    # Abrimos el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(nombreArchivo, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [4]:
dfitems = leer_json_gzip('C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/users_items.json.gz')

In [5]:
dfitems= pd.json_normalize(dfitems, record_path=['items'], meta=['user_id', 'steam_id','items_count'] )
dfitems.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,steam_id,items_count
0,10,Counter-Strike,6,0,76561197970982479,76561197970982479,277
1,20,Team Fortress Classic,0,0,76561197970982479,76561197970982479,277
2,30,Day of Defeat,7,0,76561197970982479,76561197970982479,277


In [6]:
dfitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   steam_id          object
 6   items_count       object
dtypes: int64(2), object(5)
memory usage: 275.2+ MB


In [7]:
dfitems['item_id'] = dfitems['item_id'].astype(np.int64)

In [8]:
dfitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   steam_id          object
 6   items_count       object
dtypes: int64(3), object(4)
memory usage: 275.2+ MB


In [9]:
dfitems.isna().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
steam_id            0
items_count         0
dtype: int64

In [10]:
# remove columns that not use
dfitems.drop(['playtime_2weeks'], axis=1, inplace=True)


In [11]:
dfitems.head(3)

,item_id,item_name,playtime_forever,user_id,steam_id,items_count
0,10,Counter-Strike,6,76561197970982479,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,76561197970982479,277


In [12]:
new_parquetitems_file_path = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/dfitems.parquet'

# Guardar el DataFrame en formato Parquet
dfitems.to_parquet(new_parquetitems_file_path, index=False, compression="gzip")

In [13]:
# Tomar una muestra aleatoria del 10% del dataset para la API
dfitemsAPI = dfitems.sample(frac=0.1, random_state=42)

# Guardar la muestra en la ruta especificada
ruta_salida = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/dfitemsAPI.parquet'
dfitemsAPI.to_parquet(ruta_salida, engine='pyarrow')

print(f'Se ha guardado una muestra del 10% del dataset en {ruta_salida}')


Se ha guardado una muestra del 10% del dataset en C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/dfitemsAPI.parquet
